# Day 2 - The Mallows Model

### PrefStat 2025: Second International Summer School on Preference Learning for Ranking and Ordinal Data
#### A. George, 1st July, 2025

# 0) Setup
### a) Necessary Python Installations
We work with preflibtools to upload and analyse data from PrefLib.org.
For documentation please refer to https://preflib.github.io/preflibtools/usage.html#preflib-instances.
We also use ortools in one part to compute some Maximum Likelihood parameter exactly. Appart from this some standard python pacakges for computations (numpy), data handling (pandas) and visualisations (seabors, matplotlib) are used. These packages are already installed.

### b) Importing Python Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from collections import defaultdict
from itertools import product
from ortools.linear_solver import pywraplp

from preflibtools.instances import PrefLibInstance
from preflibtools.instances import OrdinalInstance
import preflibtools.properties
from preflibtools.properties.distances import distance_matrix, spearman_footrule_distance
from preflibtools.properties.distances import kendall_tau_distance, sertel_distance

/srv/conda/envs/notebook/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### c) Loading R in Python

In [ ]:
%load_ext rpy2.ipython
from rpy2.robjects import numpy2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as robjects

/srv/conda/envs/notebook/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1185: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/home/jupyter", R: "/home/jupyter/PrefStats2025"
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1185: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/tmp/RtmpSoU4Eq", R: "/tmp/RtmpxB4nP2"
  warnings.warn(


In [ ]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))
library(BayesMallows)

* installing *source* package ‘BayesMallows’ ...
** package ‘BayesMallows’ successfully unpacked and MD5 sums checked
** using staged installation
** libs


x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/notebook/lib/R/include" -DNDEBUG  -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/Rcpp/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/RcppArmadillo/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/testthat/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /srv/conda/envs/notebook/include -I/srv/conda/envs/notebook/include -Wl,-rpath-link,/srv/conda/envs/notebook/lib  -fopenmp -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /srv/conda/envs/notebook/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1648746271917/work=/usr/local/src/conda/r-base-4.0.5 -fdebug-prefix-map=/srv/conda/envs/notebook=/usr/local/src/conda-prefix  -c RcppExports.cpp -o RcppExports.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/notebook/lib/R/i

x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/notebook/lib/R/include" -DNDEBUG  -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/Rcpp/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/RcppArmadillo/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/testthat/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /srv/conda/envs/notebook/include -I/srv/conda/envs/notebook/include -Wl,-rpath-link,/srv/conda/envs/notebook/lib  -fopenmp -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /srv/conda/envs/notebook/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1648746271917/work=/usr/local/src/conda/r-base-4.0.5 -fdebug-prefix-map=/srv/conda/envs/notebook=/usr/local/src/conda-prefix  -c importance_sampling.cpp -o importance_sampling.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/

x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/notebook/lib/R/include" -DNDEBUG  -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/Rcpp/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/RcppArmadillo/include' -I'/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/testthat/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /srv/conda/envs/notebook/include -I/srv/conda/envs/notebook/include -Wl,-rpath-link,/srv/conda/envs/notebook/lib  -fopenmp -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /srv/conda/envs/notebook/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1648746271917/work=/usr/local/src/conda/r-base-4.0.5 -fdebug-prefix-map=/srv/conda/envs/notebook=/usr/local/src/conda-prefix  -c rmallows.cpp -o rmallows.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/srv/conda/envs/notebook/lib/R/include

installing to /home/jupyter/R/x86_64-conda-linux-gnu-library/4.0/00LOCK-BayesMallows/00new/BayesMallows/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (BayesMallows)


Installing package into ‘/home/jupyter/R/x86_64-conda-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/BayesMallows_2.2.3.tar.gz'
Content type 'application/x-gzip' length 4377695 bytes (4.2 MB)
downloaded 4.2 MB


The downloaded source packages are in
	‘/tmp/RtmpXwuCYT/downloaded_packages’


-------------------------------------------
# Let's Start! :)
-------------------------------------------


# 1) Loading Data

## We begin by loading some data. Later you can choose which one you want to analyse.

### a)

We loade the Multilaps dataset (see https://preflib.github.io/PrefLib-Jekyll/dataset/00049)

*This dataset is generated from multi-lap sports competitions.
It contains the completion time of athletes in each lap of a multi-lap competition (specifically, speed skating and cycling competitions) crawled from results.sporthive.com. The data corresponds to one race in which the athletes are the candidates and each vote corresponds to one lap and ranks the athletes by their completion time. It is a post-processed version where some candidates and voters mimght have been deleted to make the election complete.
This dataset is part of a larger study by Boehmer and Schaar (see this page for a more detailed description of the dataset, the post-processing, and pointers to similar datasets). If you have any questions, please contact: niclas.boehmer@tu-berlin.de.*

In [ ]:
instance_multilaps = OrdinalInstance()
instance_multilaps.parse_url("https://raw.githubusercontent.com/PrefLib/PrefLib-Data/main/datasets/00049%20-%20mylaps/00049-00000153.soc")

### b)

We loade the t-shirt dataset (see https://preflib.github.io/PrefLib-Jekyll/dataset/00012)

*This dataset contains complete rank orderings of T-Shirt designs voted on by members of the Optimization Research Group at NICTA. There are 11 designs (candidates) and 30 votes about these deisgns. Voters were required to submit complete strict orders.*

In [ ]:
instance_shirt = OrdinalInstance()
instance_shirt.parse_url("https://raw.githubusercontent.com/PrefLib/PrefLib-Data/main/datasets/00012%20-%20shirt/00012-00000001.soc")

### c)

We loade the Sushi Data (00014) dataset (see https://preflib.github.io/PrefLib-Jekyll/dataset/00014)

*This dataset contains the results of a series of surveys conducted by Toshihiro Kamishima asking 5000 individuals for their preferences about various kinds of sushi. Element Series 00000001 contains 10 complete strict rank orders of 10 different kinds of sushi.*

In [ ]:
instance_sushi = OrdinalInstance()
instance_sushi.parse_url("https://raw.githubusercontent.com/PrefLib/PrefLib-Data/main/datasets/00014%20-%20sushi/00014-00000001.soc")

### d)

We loade the Seasons Power Ranking (00056) dataset (see https://preflib.github.io/PrefLib-Jekyll/dataset/00056)

*This dataset contains elections generated from weekly power rankings. Specifically, the underlying power ranking data (kaggle.com/masseyratings/rankings) contains weekly power rankings of college basketball teams (between 2001 and 2021), college baseball teams (between 2010 and 2021), and college American football teams (between 1997 and 2021) from different media outlets and ranking systems.*

*For each of the three sports (basketball, baseball, American football), for each season and each ranking system, we created an election where each vote corresponds to the power ranking of the teams in one week of the season according to the ranking system.*

*Each patch contains the raw election and a post-processed version where some candidates and voters are deleted to make the election complete. The name of each patch starts with the sports, followed by the relevant year and ranking system.*

*The combined power rankings and weekly power rankings datasets were generated from the same data.*

*This dataset is part of a larger study by Boehmer and Schaar (see this page for a more detailed description of the dataset, the post-processing, and pointers to similar datasets): Niclas Boehmer and Nathan Schaar. Collecting, Classifying, Analyzing, and Using Real-World Elections. Arxiv.org/abs/2204.03589, 2022 (Bibtex).*

In [ ]:
instance_power = OrdinalInstance()
instance_power.parse_url("https://raw.githubusercontent.com/PrefLib/PrefLib-Data/main/datasets/00056%20-%20seasonsport/00056-00000746.soc")

## --> Now choose an instance!

**Task**: 
1) Uncomment one of the lines below to select an instance. E.g. instance = instance_multilaps

2) Run the code and look at the number of assessors and items. What do you expect: how well can we fit a Mallows model? will we need a lot of iterations?

In [ ]:
# Choose your preferred instance here - simply comment out the others

# instance = instance_multilaps
# instance = instance_shirt
instance = instance_sushi
# instance = instance_power

# Let's look at the size of this instance
num_alts = instance.num_alternatives
num_voters = instance.num_voters

print('number of voters:', num_voters)
print('... of which unique orders:', instance.num_unique_orders)
print('number of alternatives:', num_alts)
# print('number of orders in instance.orders:', len(instance.orders))
# print('number of orders in instance.preferences:', len(instance.preferences))
# print('number of orders in instance.multiplicity:', len(instance.multiplicity))

# Get an idea of what methods are available for an instance
# by uncommenting the next line:
# dir(instance)

# Next we print the preferences of voters - just to get an impression
if num_voters > 30:
    print(f"Too many voters: {num_voters}. I will only list the first 30.")
for i, ranking in enumerate(instance.full_profile()):
    if i >= 30:
        break  # Stop after 30 iterations
    ranked_items = [item[0] for item in ranking]  # Extract item indices
    print(f"Voter {i + 1}: {' ≻ '.join(f'Item {item}' for item in ranked_items)}")


number of voters: 5000
... of which unique orders: 4926
number of alternatives: 10
Too many voters: 5000. I will only list the first 30.
Voter 1: Item 7 ≻ Item 4 ≻ Item 5 ≻ Item 1 ≻ Item 10 ≻ Item 2 ≻ Item 8 ≻ Item 3 ≻ Item 9 ≻ Item 6
Voter 2: Item 7 ≻ Item 4 ≻ Item 5 ≻ Item 1 ≻ Item 10 ≻ Item 2 ≻ Item 8 ≻ Item 3 ≻ Item 9 ≻ Item 6
Voter 3: Item 7 ≻ Item 4 ≻ Item 5 ≻ Item 1 ≻ Item 10 ≻ Item 2 ≻ Item 8 ≻ Item 3 ≻ Item 9 ≻ Item 6
Voter 4: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 8 ≻ Item 1 ≻ Item 6 ≻ Item 9
Voter 5: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 8 ≻ Item 1 ≻ Item 6 ≻ Item 9
Voter 6: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 8 ≻ Item 1 ≻ Item 6 ≻ Item 9
Voter 7: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 1 ≻ Item 8 ≻ Item 6 ≻ Item 9
Voter 8: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 1 ≻ Item 8 ≻ Item 6 ≻ Item 9
Voter 9: Item 4 ≻ Item 5 ≻ Item 7 ≻ Item 2 ≻ Item 10 ≻ Item 3 ≻ Item 1 ≻ Item 8

### 1) Kendall Tau Distance

In [ ]:
assert num_voters <= 30, f"Too many voters: {num_voters}. I will not compute the distance-heatmap."

# Computing the distances between all pairs of voters
arr_KTdist = np.array(distance_matrix(instance, kendall_tau_distance))

# 1. Maximal distance
max_val = np.max(arr_KTdist)
print("Max distance between voters:", max_val)

# 2. Average distance (excluding diagonal)
# Mask out diagonal
off_diagonal_mask = ~np.eye(arr_KTdist.shape[0], dtype=bool)
off_diagonal_values = arr_KTdist[off_diagonal_mask]
average_off_diagonal = np.mean(off_diagonal_values)
print("Average distance between voters:", average_off_diagonal)

# 3. Plotting a heatmap to show all distancesbetween pairs of voters
plt.figure(figsize=(12, 10))
sns.heatmap(arr_KTdist, annot=True, fmt=".0f", cmap="coolwarm", cbar=True)
plt.title("Kendall Tau Distances Between Voters")
plt.xlabel("Voter")
plt.ylabel("Voter")
plt.show()

AssertionError: Too many voters: 5000. I will not compute the distance-heatmap.

.... TODO compute the normalising instance (efficiently)

### 2) Spearman Distance

In [ ]:
assert num_voters <= 30, f"Too many voters: {num_voters}. I will not compute the distance-heatmap."

# Computing the distances between all pairs of voters
arr = np.array(distance_matrix(instance, spearman_footrule_distance))

# 1. Maximal distance
max_val = np.max(arr)
print("Max distance between voters:", max_val)

# 2. Average distance (excluding diagonal)
# Mask out diagonal
off_diagonal_mask = ~np.eye(arr.shape[0], dtype=bool)
off_diagonal_values = arr[off_diagonal_mask]
average_off_diagonal = np.mean(off_diagonal_values)
print("Average distance between voters:", average_off_diagonal)

# 3. Plotting a heatmap to show all distancesbetween pairs of voters
plt.figure(figsize=(12, 10))
sns.heatmap(arr, annot=True, fmt=".0f", cmap="coolwarm", cbar=True)
plt.title("Spearman Distances Between Voters")
plt.xlabel("Voter")
plt.ylabel("Voter")
plt.show()

.... TODO compute the normalising instance (efficiently)

Question: Given the heatmap (differences between voters) and the maximal and average difference - what are your expectations for the Mallows model?

## Maximum Liklihood Models

### 1) Mallows Model with Kendall Tau Distance

In [ ]:

# Initialize the agreement matrix Q with zeros
Q = np.zeros((num_alts, num_alts), dtype=int)

# For each preference we consider the ranking 
# and count is the number of voters with that ranking
for ind, ranking in enumerate(instance.full_profile()):
    for i in range(len(ranking)):
        for j in range(i + 1, len(ranking)):
            (higher,) = ranking[i]
            (lower,) = ranking[j]
            # Update the number of voters that prefer higher to lower
            Q[int(higher) - 1, int(lower) - 1] += 1

# print(Q)
# Create the MIP solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define 0,1 variables for every ordered pair of items
x = {}
for i in range(num_alts):
    for j in range(num_alts):
        x[i, j] = solver.IntVar(0, 1, f'x_{i}_{j}')

# Define Constraints ensuring only one tuple (a,b) or (b,a) is selected: 
# x_i_j + x_j_i = 1
for i, j in itertools.combinations(range(num_alts), 2):
    constraint = solver.RowConstraint(1, 1)
    constraint.SetCoefficient(x[i, j], 1)
    constraint.SetCoefficient(x[j, i], 1)

# Define constraints ensuring transitivity: 
# 1 <= x_i_j + x_j_h + x_h_i <= 2
for i, j, h in itertools.combinations(range(num_alts), 3):
    constraint = solver.RowConstraint(1, 2)
    constraint.SetCoefficient(x[i, j], 1)
    constraint.SetCoefficient(x[j, h], 1)
    constraint.SetCoefficient(x[h, i], 1)

# Define objective
objective = solver.Objective()
for i in range(num_alts):
    for j in range(num_alts):
        if i != j:
            objective.SetCoefficient(x[i, j], int(Q[j][i]))
objective.SetMinimization()

status = solver.Solve()

# Set Kemeny Score and Kemeny Ranking from ILP solution
kemeny_score = solver.Objective().Value()
kemeny_ranking = -1 * np.ones(num_alts)
for i in range(num_alts):
    sum = 0
    # for every item compute the number of lower ranked item (in "sum")
    for j in range(num_alts):  
        if i != j:
            sum += x[i, j].solution_value()
    # the item i is placed according to the number of lower ranked items
    kemeny_ranking[int(num_alts - sum - 1)] = i  

# Print the MLE ranking
print("\nThe maximum likelihood ranking for the Mallows model with Kendall Tau distance is:")
print(" ≻ ".join(str(int(x+1)) for x in kemeny_ranking))
print("\nThe total Kendall Tau distance w.r.t. the voters is:", kemeny_score)

### 2) Mallows Model with Spearman Distance

In [ ]:
borda_scores = defaultdict(float)

# Compute Borda Scores
for ranking in instance.full_profile():
    for i, alt_group in enumerate(ranking):
        if isinstance(alt_group, int):
            borda_scores[alt_group] += num_alts - i - 1
        else:  # Tied alternatives
            for alt in alt_group:
                borda_scores[alt] += num_alts - i - 1

# Sort and display
sorted_scores = sorted(borda_scores.items(), key=lambda x: x[1], reverse=True)
for alt, score in sorted_scores:
    print(f"Alternative {alt}: {score:.1f} points")

# Print the MLE ranking
borda_ranking = [f"{alt}" for alt, _ in sorted_scores]
print("\nThe maximum likelihood ranking for the Mallows model with Spearman distance is:")
print(" ≻ ".join(borda_ranking))

# Using BayesMallows in R

## 1) Transforming the data

In [ ]:
# Each preference in instance.preferences is a tuple of tuples of item IDs
# Convert to numpy matrix: rows = assessors, columns = ranked items

rankings_matrix = np.zeros((num_voters, num_alts), dtype=int)

for i, ranking in enumerate(instance.full_profile()):
    for rank_pos, item_tuple in enumerate(ranking):
        item = item_tuple[0]
#         rankings_matrix[i, rank_pos] = item
        # item is 1-based in PrefLib; rank is 0-based → add 1 to rank to get 1-based rank for R
        rankings_matrix[i, item - 1] = rank_pos + 1

# print(rankings_matrix)

In [ ]:
with localconverter(robjects.default_converter + numpy2ri.converter):
    r_matrix = robjects.conversion.py2rpy(rankings_matrix)

robjects.globalenv['rankings_matrix'] = r_matrix
robjects.globalenv['n_assessors'] = num_voters
robjects.globalenv['n_items'] = num_alts


# Exercise: Compare different distance metrics
BayesMallows supports different distances (e.g., Kendall, Footrule, Spearman). You can fit Mallows models using different metrics and compare outputs like consensus ranking, posterior uncertainty, and goodness-of-fit.

Task:
Fit Mallows models with Kendall, Footrule, and Spearman distances on the same dataset. Compare consensus rankings and convergence diagnostics. Which metric fits the data best?

In [ ]:
%%R
library(BayesMallows)

set.seed(456)

# Step 1: Convert numpy matrix to BayesMallowsData object
data <- setup_rank_data(rankings_matrix)
# print(data)

metrics <- c("footrule", "spearman", "cayley", "hamming", "ulam", "kendall")

results <- data.frame(
  metric = character(),
  alpha = numeric(),
  consensus = character(),
  stringsAsFactors = FALSE
)

for (m in metrics) {
  cat("\nFitting Mallows model with metric:", m, "\n")
  
  model_options <- set_model_options(metric = m, n_clusters = 1)
  compute_options <- set_compute_options(nmc = 5000, burnin = 1000, rho_thinning = 10)
# nmc: number of total iterations
# burnin: how many samples to discard from the beginning
# rho_thinning: how many samples to skip between retained samples
  model <- compute_mallows(data, model_options = model_options, compute_options = compute_options)
  
#   print(summary(model))
    
  # Estimate alpha from posterior samples
  alpha_val <- mean(model$alpha$value)
  
  # Estimate consensus ranking from rho samples
  rho_means <- aggregate(value ~ item, data = model$rho, FUN = mean)
  ordered_items <- rho_means$item[order(rho_means$value)]
  consensus_str <- paste(ordered_items, collapse = " ≻ ")
  
  # Store results
  results <- rbind(
#     results,
    data.frame(metric = m, alpha = alpha_val, consensus = consensus_str)
  )
}

print(results)

In [ ]:
from rpy2.robjects import r, default_converter
from rpy2.robjects.conversion import localconverter

# Extract ordered_items from R using modern conversion
with localconverter(default_converter):
    ordered_items_r = r('rho_means$item[order(rho_means$value)]')

# Convert directly to Python list of integers
Mallows_consensus = list(ordered_items_r)

# Convert Kemeny ranking from before to the same format (list of ints that has values from 1 to 11)
KT_minimising_ranking = [int(x)+1 for x in kemeny_ranking]

# Then compute Kemeny score as before (assuming Q is your numpy matrix)
def kemeny_score_from_Q(Q, ranking):
    score = 0
    n = len(ranking)
    for i in range(n - 1):
        for j in range(i + 1, n):
            h = ranking[i] - 1  # zero-based index for Q
            l = ranking[j] - 1
            score += Q[l, h]  # count inversions: how many voters prefer l over h
    return score

score = kemeny_score_from_Q(Q, Mallows_consensus)
print("Consensus ranking (Mallows_consensus):\n", Mallows_consensus)
print(f"Kemeny score of consensus ranking: {score}")

score = kemeny_score_from_Q(Q, KT_minimising_ranking)
print("KT minimising ranking:\n", KT_minimising_ranking)
print(f"Kemeny score of KT_minimising_ranking: {score}")

In [ ]:
print("\nThe maximum likelihood ranking for the Mallows model with Spearman distance is:")
print(" ≻ ".join(borda_ranking))

2. Goodness-of-fit and model checking
Teach students to assess model fit by examining posterior distributions, traceplots, and how well the model recovers known rankings (if simulated data).

Plot posterior distributions of the consensus ranking

Posterior predictive checks (simulate rankings from the fitted model and compare to observed)

In [ ]:
%%R

library(ggplot2)

ggplot(model$alpha, aes(x=value)) +
  geom_histogram(bins=30, fill="skyblue", color="black") +
  ggtitle(paste("Posterior distribution of alpha - Metric:", model_options$metric)) +
  xlab("Alpha (dispersion)") +
  ylab("Frequency")


In [ ]:
%%R

ggplot(model$alpha, aes(x=iteration, y=value)) +
  geom_line() +
  ggtitle("Trace plot of alpha samples") +
  xlab("Iteration") + ylab("Alpha")

# 3. Clustering ranking data

Real ranking data often comes from heterogeneous populations. 
BayesMallows supports mixture models to cluster assessors into subgroups with different consensus rankings.

**Task**:
Fit a Mallows mixture model with 2 or 3 clusters, interpret clusters, and compare consensus rankings within each cluster. 
How many clusters should we choose?

In [ ]:
%%R

model_options <- set_model_options(metric = "kendall", n_clusters = 2)
model <- compute_mallows(data, model_options = model_options)

# Extract consensus for each cluster
for (clust in levels(model$rho$cluster)) {
  rho_cluster <- subset(model$rho, cluster == clust)
  rho_means <- aggregate(value ~ item, data = rho_cluster, FUN = mean)
  ordered_items <- rho_means$item[order(rho_means$value)]
  cat("Consensus ranking for cluster", clust, ":\n", paste(ordered_items, collapse = " ≻ "), "\n")
}

# Cluster assignment counts
table(model$cluster_assignment$value)
